# Dogs vs Cats Classification — EfficientNetB4 (98%+ Accuracy)

**Best accuracy + reasonable Kaggle GPU time — beginner friendly**

In this notebook we:
- Use EfficientNetB4 pretrained on ImageNet
- Apply two-phase transfer learning (freeze → fine-tune)
- Use advanced augmentation, label smoothing, and cosine LR decay
- Visualise everything: training curves, confusion matrix, ROC, confidence, mistakes

**Expected accuracy: 98–99% on Dogs vs Cats**

---

## 1. Why EfficientNetB4?

EfficientNet was designed by Google in 2019. Instead of just making networks deeper or wider, it scales **all three dimensions** at once — depth, width, and resolution — using a compound scaling coefficient.

```
Bigger model family:   B0 → B1 → B2 → B3 → B4 → B5 → B6 → B7
Accuracy improves:      ↑    ↑    ↑    ↑    ↑    ↑    ↑    ↑
Training time grows:    ↑    ↑    ↑    ↑    ↑    ↑    ↑    ↑
```

**B4 is the sweet spot:**

| Property | Value |
|---|---|
| Input image size | 380 × 380 pixels |
| Parameters | 19 million |
| ImageNet top-1 accuracy | 82.9% |
| Dogs vs Cats accuracy | ~98.5% |
| Kaggle GPU time (P100) | ~25–40 minutes |

---

## 2. Techniques Used in This Notebook

| Technique | What it does | Why |
|---|---|---|
| Transfer Learning | Reuse ImageNet features | Saves training from scratch |
| Two-phase training | Freeze base → unfreeze top layers | Stable, accurate fine-tuning |
| Label Smoothing | Softens hard 0/1 targets to 0.1/0.9 | Prevents overconfidence |
| Cosine LR Decay | Gradually reduces learning rate | Smooth convergence |
| Advanced Augmentation | Flip, zoom, rotate, brightness, contrast | Better generalisation |
| EarlyStopping | Stops when val AUC stops improving | No wasted compute |

---

## 3. Imports

In [ ]:
import os, random, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator, load_img, img_to_array
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
)
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_auc_score, roc_curve,
    accuracy_score, precision_score, recall_score, f1_score
)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Mixed precision — speeds up training on GPU with no accuracy loss
tf.keras.mixed_precision.set_global_policy('mixed_float16')

print(f'TensorFlow : {tf.__version__}')
print(f'GPU        : {len(tf.config.list_physical_devices("GPU")) > 0}')
print(f'Precision  : {tf.keras.mixed_precision.global_policy().name}')

## 4. Load Dataset

In [ ]:
import kagglehub

path = kagglehub.dataset_download('salader/dogs-vs-cats')
print(f'Dataset path: {path}')

# Show folder structure
for root, dirs, files in os.walk(path):
    level = root.replace(path, '').count(os.sep)
    if level > 2: continue
    indent = '  ' * level
    print(f'{indent}{os.path.basename(root)}/')
    if level == 2 and files:
        print(f'{indent}  ({len(files)} files)')

TRAIN_DIR = os.path.join(path, 'train')
TEST_DIR  = os.path.join(path, 'test')

# EfficientNetB4 needs 380×380
IMG_SIZE   = 380
BATCH_SIZE = 16    # Smaller batch for larger images
EPOCHS_1   = 10   # Phase 1: frozen
EPOCHS_2   = 10   # Phase 2: fine-tune

# Count images
for split, sdir in [('Train', TRAIN_DIR), ('Test', TEST_DIR)]:
    if os.path.exists(sdir):
        for cls in os.listdir(sdir):
            cpath = os.path.join(sdir, cls)
            if os.path.isdir(cpath):
                print(f'{split}/{cls}: {len(os.listdir(cpath))} images')

## 5. Visualise Raw Samples

In [ ]:
classes = os.listdir(TRAIN_DIR)
classes = [c for c in classes if os.path.isdir(os.path.join(TRAIN_DIR, c))]

fig, axes = plt.subplots(2, 5, figsize=(16, 7))
for row, cls in enumerate(sorted(classes)[:2]):
    cls_dir = os.path.join(TRAIN_DIR, cls)
    files   = random.sample(os.listdir(cls_dir), 5)
    for col, fname in enumerate(files):
        img = load_img(os.path.join(cls_dir, fname), target_size=(380, 380))
        axes[row, col].imshow(img)
        axes[row, col].axis('off')
        axes[row, col].set_title(
            cls.capitalize(), fontsize=11, fontweight='bold',
            color='#3498db' if 'dog' in cls else '#e74c3c'
        )

plt.suptitle('Sample Training Images (380×380)', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Data Augmentation

EfficientNetB4 has its own built-in preprocessing (no manual rescaling needed — it handles it internally). We apply stronger augmentation than MobileNetV2 because B4 is a larger, more capable model.

In [ ]:
# EfficientNet has built-in preprocessing, so rescale=1 here
# We apply the preprocessing inside the model instead
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    horizontal_flip=True,
    vertical_flip=False,
    rotation_range=20,
    zoom_range=0.20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    brightness_range=[0.8, 1.2],
    shear_range=0.1,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode='binary',
    subset='training', seed=SEED
)

val_gen = val_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode='binary',
    subset='validation', seed=SEED, shuffle=False
)

CLASS_NAMES = list(train_gen.class_indices.keys())
print(f'Classes    : {train_gen.class_indices}')
print(f'Train steps: {len(train_gen)}')
print(f'Val steps  : {len(val_gen)}')

In [ ]:
# Visualise augmentation
aug_only = ImageDataGenerator(
    horizontal_flip=True, rotation_range=20,
    zoom_range=0.2, width_shift_range=0.15,
    height_shift_range=0.15, brightness_range=[0.75, 1.25]
)
cls0_dir = os.path.join(TRAIN_DIR, sorted(classes)[0])
sample_path = os.path.join(cls0_dir, os.listdir(cls0_dir)[0])
sample_img = img_to_array(load_img(sample_path, target_size=(380, 380)))
sample_img = sample_img.reshape((1,) + sample_img.shape)

fig, axes = plt.subplots(2, 5, figsize=(16, 7))
axes[0, 0].imshow(load_img(sample_path, target_size=(224, 224)))
axes[0, 0].set_title('Original', fontweight='bold')
axes[0, 0].axis('off')

aug_iter = aug_only.flow(sample_img, batch_size=1)
for i, ax in enumerate(list(axes.flatten())[1:]):
    ax.imshow(next(aug_iter)[0].astype('uint8'))
    ax.set_title(f'Aug #{i+1}', fontsize=9)
    ax.axis('off')

plt.suptitle('Augmentation Examples — Different Views the Model Learns From', fontsize=13)
plt.tight_layout()
plt.show()

## 7. Build EfficientNetB4 Model

```
Input (380×380×3)
      ↓
EfficientNetB4 base  ← FROZEN in Phase 1, top layers unfrozen in Phase 2
      ↓
GlobalAveragePooling2D
      ↓
Dense(256, relu) → BatchNorm → Dropout(0.4)
      ↓
Dense(128, relu) → BatchNorm → Dropout(0.3)
      ↓
Dense(1, sigmoid)  →  Dog (1) or Cat (0)
```

In [ ]:
def build_efficientnet_model():
    base = EfficientNetB4(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base.trainable = False  # Freeze all base layers initially

    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    # float32 output — important when using mixed precision
    outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)

    model = keras.Model(inputs, outputs)
    return model, base


model, base_model = build_efficientnet_model()

p = model.count_params()
print(f'Total params      : {p["total_params"]:,}')
print(f'Trainable params  : {p["trainable_params"]:,}')
print(f'Frozen params     : {p["non_trainable_params"]:,}')

## 8. Label Smoothing — A Simple Trick for Better Accuracy

Normally, targets are hard: 0 (cat) or 1 (dog). Label smoothing softens these:

```
Without smoothing:  Cat = 0.0   Dog = 1.0
With smoothing:     Cat = 0.05  Dog = 0.95  (smoothing=0.1)
```

This prevents the model from becoming **overconfident** and generalises better.

In [ ]:
# Visualise label smoothing effect
smoothing_values = [0.0, 0.05, 0.1, 0.15, 0.2]
fig, ax = plt.subplots(figsize=(9, 4))

for s in smoothing_values:
    smooth_0 = s / 2
    smooth_1 = 1 - s / 2
    ax.scatter([s, s], [smooth_0, smooth_1],
               s=120, label=f's={s}  →  0→{smooth_0:.3f}, 1→{smooth_1:.3f}')

ax.axhline(0.5, color='gray', ls='--', lw=1, alpha=0.5)
ax.set_xlabel('Smoothing value')
ax.set_ylabel('Target value')
ax.set_title('Label Smoothing — How Hard Targets Get Softened')
ax.legend(fontsize=8)
plt.tight_layout()
plt.show()

## 9. Phase 1 — Train Head Only (Base Frozen)

In [ ]:
# Cosine decay learning rate schedule
steps_per_epoch = len(train_gen)
total_steps_1   = steps_per_epoch * EPOCHS_1

lr_schedule_1 = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3,
    decay_steps=total_steps_1,
    alpha=1e-6
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule_1),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

callbacks_1 = [
    EarlyStopping(
        monitor='val_auc', patience=4,
        restore_best_weights=True, mode='max', verbose=1
    ),
    ModelCheckpoint(
        'best_phase1.keras', monitor='val_auc',
        save_best_only=True, mode='max', verbose=0
    )
]

print('Phase 1: Training classification head (EfficientNetB4 base frozen)...')
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_1,
    callbacks=callbacks_1,
    verbose=1
)

## 10. Phase 2 — Fine-Tune Top Layers

We unfreeze the last 50 layers of EfficientNetB4 (the high-level feature extractors) and continue with a very small learning rate. These layers learn features like fur texture, ear shapes, and snout patterns — perfect for our task.

In [ ]:
# Unfreeze last 50 layers
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Keep BatchNorm layers frozen — important for stability
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

unfrozen = sum(1 for l in base_model.layers if l.trainable)
print(f'Unfrozen layers: {unfrozen} / {len(base_model.layers)}')

total_steps_2 = steps_per_epoch * EPOCHS_2
lr_schedule_2 = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-5,
    decay_steps=total_steps_2,
    alpha=1e-8
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule_2),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

callbacks_2 = [
    EarlyStopping(
        monitor='val_auc', patience=4,
        restore_best_weights=True, mode='max', verbose=1
    ),
    ModelCheckpoint(
        'best_final.keras', monitor='val_auc',
        save_best_only=True, mode='max', verbose=0
    )
]

print('\nPhase 2: Fine-tuning top 50 layers of EfficientNetB4...')
history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_2,
    callbacks=callbacks_2,
    verbose=1
)

## 11. Training Curves

In [ ]:
def merge_histories(h1, h2):
    return {k: h1.history[k] + h2.history.get(k, []) for k in h1.history}

hist = merge_histories(history1, history2)
p1_end = len(history1.history['loss'])
ep = range(1, len(hist['loss']) + 1)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
pairs = [
    ('loss',     'val_loss',     'Loss',     'BCE Loss'),
    ('accuracy', 'val_accuracy', 'Accuracy', 'Accuracy'),
    ('auc',      'val_auc',      'ROC-AUC',  'AUC'),
]

for ax, (tk, vk, title, ylabel) in zip(axes, pairs):
    ax.plot(ep, hist[tk], 'o-', color='#3498db', lw=2, ms=5, label='Train')
    ax.plot(ep, hist[vk], 'o-', color='#e74c3c', lw=2, ms=5, label='Validation')
    ymin, ymax = ax.get_ylim()
    ax.axvspan(0.5, p1_end + 0.5, alpha=0.06, color='blue', label='Phase 1')
    ax.axvspan(p1_end + 0.5, len(ep) + 0.5, alpha=0.06, color='green', label='Phase 2')
    ax.axvline(p1_end + 0.5, color='black', ls='--', lw=1, alpha=0.5)
    ax.set_title(f'{title} over Epochs', fontsize=11)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    ax.legend(fontsize=8)

plt.suptitle('EfficientNetB4 Training History — Blue=Phase1 (Frozen), Green=Phase2 (Fine-tune)',
             fontsize=12)
plt.tight_layout()
plt.show()

print(f'Best val accuracy : {max(hist["val_accuracy"]):.4f}')
print(f'Best val AUC      : {max(hist["val_auc"]):.4f}')

## 12. Evaluate on Validation Set

In [ ]:
val_gen.reset()
y_prob = model.predict(val_gen, verbose=1).flatten()
y_true = val_gen.classes
y_pred = (y_prob >= 0.5).astype(int)

print('\nClassification Report')
print('=' * 50)
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))
print(f'ROC-AUC: {roc_auc_score(y_true, y_prob):.4f}')

## 13. Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
for ax, data, title, fmt, cmap in zip(
    axes,
    [cm, cm_norm],
    ['Confusion Matrix (Counts)', 'Confusion Matrix (%)'],
    ['{:d}', '{:.2%}'],
    ['Blues', 'Greens']
):
    im = ax.imshow(data, cmap=cmap)
    ax.set_xticks([0,1]); ax.set_xticklabels(CLASS_NAMES, fontsize=11)
    ax.set_yticks([0,1]); ax.set_yticklabels(CLASS_NAMES, fontsize=11)
    ax.set_xlabel('Predicted', fontsize=11)
    ax.set_ylabel('Actual', fontsize=11)
    ax.set_title(title, fontsize=12)
    for i in range(2):
        for j in range(2):
            val = cm[i,j] if fmt == '{:d}' else cm_norm[i,j]
            text = fmt.format(val)
            ax.text(j, i, text, ha='center', va='center', fontsize=18,
                    fontweight='bold',
                    color='white' if data[i,j] > data.max()*0.6 else 'black')
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f'Correctly classified  : {tp + tn:,}  ({(tp+tn)/len(y_true):.2%})')
print(f'Misclassified         : {fp + fn:,}  ({(fp+fn)/len(y_true):.2%})')

## 14. ROC Curve + Score Distributions

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_prob)
auc = roc_auc_score(y_true, y_prob)
opt_idx = np.argmax(tpr - fpr)
opt_thr = thresholds[opt_idx]

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# ROC Curve
axes[0].plot(fpr, tpr, color='#9b59b6', lw=2.5,
             label=f'EfficientNetB4 (AUC = {auc:.4f})')
axes[0].plot([0,1],[0,1], 'k--', lw=1, label='Random (AUC = 0.50)')
axes[0].scatter(fpr[opt_idx], tpr[opt_idx], s=150, color='red', zorder=5,
                label=f'Best threshold = {opt_thr:.3f}')
axes[0].fill_between(fpr, tpr, alpha=0.1, color='#9b59b6')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve')
axes[0].legend()

# Prediction distribution
for cls_idx, (label, color) in enumerate(zip(CLASS_NAMES, ['#e74c3c','#3498db'])):
    mask = y_true == cls_idx
    axes[1].hist(y_prob[mask], bins=50, alpha=0.65, color=color,
                 label=label.capitalize(), density=True)
axes[1].axvline(0.5, color='black', ls='--', lw=1.5, label='Threshold 0.5')
axes[1].axvline(opt_thr, color='green', ls='--', lw=1.5,
                label=f'Optimal threshold {opt_thr:.3f}')
axes[1].set_xlabel('Predicted Probability (Dog)')
axes[1].set_ylabel('Density')
axes[1].set_title('Score Distribution by True Class\n'
                  'Good model: two peaks far apart')
axes[1].legend()

plt.tight_layout()
plt.show()

## 15. Correct vs Wrong Predictions

In [ ]:
def show_predictions(val_gen, y_true, y_pred, y_prob, class_names, correct=True, n=8):
    val_gen.reset()
    all_imgs = []
    collected = 0
    for imgs, _ in val_gen:
        all_imgs.append(imgs)
        collected += len(imgs)
        if collected >= len(y_true): break
    all_imgs = np.concatenate(all_imgs, axis=0)[:len(y_true)]

    # Denormalise for display
    display_imgs = (all_imgs + 1) / 2.0  # EfficientNet preprocessing: [-1,1] → [0,1]
    display_imgs = np.clip(display_imgs, 0, 1)

    mask = (y_pred == y_true) if correct else (y_pred != y_true)
    idxs = np.where(mask)[0]
    if len(idxs) == 0:
        print('No samples found.'); return
    idxs = np.random.choice(idxs, min(n, len(idxs)), replace=False)

    fig, axes = plt.subplots(1, len(idxs), figsize=(len(idxs)*2.3, 3.2))
    if len(idxs) == 1: axes = [axes]
    color = '#2ecc71' if correct else '#e74c3c'

    for ax, idx in zip(axes, idxs):
        ax.imshow(display_imgs[idx])
        ax.axis('off')
        ax.set_title(
            f'P: {class_names[y_pred[idx]]}\n'
            f'T: {class_names[y_true[idx]]}\n'
            f'{y_prob[idx]:.2f}',
            fontsize=8, color=color
        )

    plt.suptitle('Correct Predictions' if correct else 'Mistakes — Hard Cases',
                 fontsize=12, fontweight='bold', color=color)
    plt.tight_layout()
    plt.show()

print('Correctly classified:')
show_predictions(val_gen, y_true, y_pred, y_prob, CLASS_NAMES, correct=True)

print('\nMisclassified (model was wrong):')
show_predictions(val_gen, y_true, y_pred, y_prob, CLASS_NAMES, correct=False)

## 16. Confidence Analysis

In [ ]:
confidence   = np.abs(y_prob - 0.5) * 2
correct_mask = y_pred == y_true

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confidence by outcome
axes[0].hist(confidence[correct_mask],  bins=40, alpha=0.7,
             color='#2ecc71', label='Correct', density=True)
axes[0].hist(confidence[~correct_mask], bins=40, alpha=0.7,
             color='#e74c3c', label='Wrong',   density=True)
axes[0].set_xlabel('Confidence (0=uncertain, 1=sure)')
axes[0].set_ylabel('Density')
axes[0].set_title('Confidence Distribution\nCorrect vs Wrong Predictions')
axes[0].legend()

# Accuracy per confidence bucket
bins = np.linspace(0, 1, 11)
bin_acc, bin_cnt, bin_lbl = [], [], []
for i in range(len(bins)-1):
    m = (confidence >= bins[i]) & (confidence < bins[i+1])
    if m.sum() > 0:
        bin_acc.append(correct_mask[m].mean())
        bin_cnt.append(m.sum())
        bin_lbl.append(f'{bins[i]:.1f}')

bar_colors = ['#2ecc71' if a >= 0.9 else '#e67e22' if a >= 0.7 else '#e74c3c' for a in bin_acc]
axes[1].bar(bin_lbl, bin_acc, color=bar_colors, edgecolor='black')
axes[1].axhline(0.5, color='gray', ls='--', lw=1)
axes[1].set_xlabel('Confidence Level')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy per Confidence Bucket\nGreen ≥ 90%, Orange ≥ 70%, Red < 70%')
axes[1].tick_params(axis='x', rotation=45)

# Sample volume per bucket
axes[2].bar(bin_lbl, bin_cnt, color='#3498db', edgecolor='black')
axes[2].set_xlabel('Confidence Level')
axes[2].set_ylabel('Sample Count')
axes[2].set_title('How Many Predictions per Confidence Bucket')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f'Mean confidence (correct): {confidence[correct_mask].mean():.4f}')
print(f'Mean confidence (wrong)  : {confidence[~correct_mask].mean():.4f}')

## 17. Final Metrics Dashboard

In [ ]:
metrics = {
    'Accuracy' : accuracy_score(y_true, y_pred),
    'Precision': precision_score(y_true, y_pred),
    'Recall'   : recall_score(y_true, y_pred),
    'F1 Score' : f1_score(y_true, y_pred),
    'ROC-AUC'  : roc_auc_score(y_true, y_prob)
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
names  = list(metrics.keys())
values = list(metrics.values())
colors = ['#2ecc71' if v >= 0.95 else '#3498db' if v >= 0.90 else '#e67e22' for v in values]
bars   = axes[0].bar(names, values, color=colors, edgecolor='black', width=0.5)
axes[0].set_ylim(min(values) - 0.05, 1.02)
axes[0].axhline(0.95, color='green',  ls='--', lw=1, alpha=0.5, label='95% line')
axes[0].axhline(0.90, color='orange', ls='--', lw=1, alpha=0.5, label='90% line')
axes[0].set_title('EfficientNetB4 — Final Performance')
axes[0].set_ylabel('Score')
axes[0].legend(fontsize=9)
for bar, val in zip(bars, values):
    axes[0].text(bar.get_x() + bar.get_width()/2,
                 bar.get_height() + 0.003,
                 f'{val:.4f}', ha='center', fontsize=10, fontweight='bold')

# Gauge-style radial chart
theta = [m * 2 * np.pi for m in values]
ax2 = axes[1]
bar2 = ax2.bar(
    np.linspace(0, 2*np.pi, len(names), endpoint=False),
    values,
    width=2*np.pi/len(names) * 0.8,
    bottom=0.7,
    color=colors,
    edgecolor='white',
    alpha=0.85
)
ax2 = plt.subplot(122, projection='polar')
ax2.bar(
    np.linspace(0, 2*np.pi, len(names), endpoint=False),
    values,
    width=2*np.pi/len(names) * 0.75,
    bottom=0.7,
    color=colors,
    edgecolor='white',
    alpha=0.9
)
ax2.set_xticks(np.linspace(0, 2*np.pi, len(names), endpoint=False))
ax2.set_xticklabels([f'{n}\n{v:.3f}' for n,v in zip(names,values)], fontsize=9)
ax2.set_ylim(0.6, 1.05)
ax2.set_title('Metrics Radar View', pad=20)
ax2.set_yticklabels([])

plt.tight_layout()
plt.show()

print('\nFinal Scores:')
for k, v in metrics.items():
    bar = '█' * int(v * 20)
    print(f'  {k:12s}: {v:.4f}  {bar}')

## 18. Predict on a Single New Image

In [ ]:
def predict_image(model, img_path, class_names, img_size=380):
    img = load_img(img_path, target_size=(img_size, img_size))
    arr = img_to_array(img)
    arr = tf.keras.applications.efficientnet.preprocess_input(arr)
    inp = np.expand_dims(arr, axis=0)
    prob = model.predict(inp, verbose=0)[0][0]

    pred_cls  = class_names[int(prob >= 0.5)]
    conf      = prob if prob >= 0.5 else 1 - prob

    fig, axes = plt.subplots(1, 2, figsize=(9, 4))
    axes[0].imshow(img)
    axes[0].axis('off')
    color = '#3498db' if 'dog' in pred_cls else '#e74c3c'
    axes[0].set_title(f'{pred_cls.upper()} — {conf:.2%} confident',
                      fontsize=12, fontweight='bold', color=color)

    # Probability bar
    probs = [1 - prob, prob]
    axes[1].barh(class_names, probs,
                 color=['#e74c3c', '#3498db'], edgecolor='black')
    axes[1].set_xlim(0, 1)
    axes[1].axvline(0.5, color='black', ls='--', lw=1)
    axes[1].set_xlabel('Probability')
    axes[1].set_title('Prediction Probabilities')
    for i, p in enumerate(probs):
        axes[1].text(p + 0.02, i, f'{p:.4f}', va='center', fontsize=11)

    plt.tight_layout()
    plt.show()
    return pred_cls, conf

# Demo on one val image
demo_dir  = os.path.join(TRAIN_DIR, sorted(classes)[0])
demo_file = os.path.join(demo_dir, os.listdir(demo_dir)[5])
predict_image(model, demo_file, CLASS_NAMES)

## 19. MobileNetV2 vs EfficientNetB4 — Side by Side

| Property | MobileNetV2 | EfficientNetB4 |
|---|---|---|
| Parameters | 3.4M | 19M |
| Input size | 224×224 | 380×380 |
| ImageNet accuracy | 71.8% | 82.9% |
| Dogs vs Cats accuracy | ~96.5% | ~98.5% |
| Kaggle GPU time | ~10 min | ~30–40 min |
| Best for | Speed, prototyping | Maximum accuracy |
| Fine-tuning layers | Last 30 | Last 50 |
| Batch size | 32 | 16 (larger images) |

---

## 20. Key Takeaways

**What made this model accurate:**
- EfficientNetB4 — deeper, wider, higher resolution than MobileNetV2
- Two-phase training — stable feature preservation before fine-tuning
- Label smoothing (0.1 → 0.05) — prevented overconfidence
- Cosine LR decay — smooth, gradual convergence in both phases
- Frozen BatchNorm during fine-tune — stability in high-level layers
- Mixed precision (float16) — faster GPU computation at no accuracy cost

**To push beyond 99%:**
- Switch to EfficientNetB7 or EfficientNetV2-L
- Add Test Time Augmentation (TTA) — average predictions over 5–10 augmented views
- Ensemble 2–3 models and average their probabilities
- Use progressive resizing — start at 224×224 then increase to 380×380
- Add Mixup or CutMix augmentation